In [1]:
import pandas as pd
import numpy as np

import keras
import keras.backend as K
from keras import layers, models, optimizers
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import Huber

from sklearn.preprocessing import LabelEncoder

import gc
import os



Using TensorFlow backend.


In [2]:
tr = pd.read_csv('./data/train.csv')
te = pd.read_csv('./data/test.csv')
trr = tr.copy()
sub = pd.read_csv('./data/sample_submission.csv')

In [3]:
train_X = tr.drop(['id', 'type'], axis=1)
test_X = te.drop(['id'], axis=1)

target = tr['type']


In [4]:
target_lbe = LabelEncoder().fit(target)

t = target_lbe.transform(target)

In [5]:
train_X['index'] = np.ones(len(train_X))
test_X['index'] = np.ones(len(test_X))*2

merge = pd.concat([train_X, test_X], ignore_index=True)

k = train_X.columns[1:-1]

In [6]:
merge = pd.concat([merge, pd.get_dummies(merge['fiberID'], prefix='fiberID')], axis=1)
merge = merge.drop('fiberID', axis=1)


In [7]:
train_X = merge[merge['index'] == 1]
train_X = train_X.drop('index', axis=1)
test_X = merge[merge['index'] == 2]
test_X = test_X.drop('index', axis=1)
test_X.index = range(len(test_X))

In [8]:
tr_X = train_X[k]
te_X = test_X[k]

tr_X = (tr_X - np.mean(tr_X))/np.std(tr_X)
te_X = (te_X - np.mean(te_X))/np.std(te_X)

train_X[k] = tr_X
test_X[k] = te_X

In [28]:
es = EarlyStopping(patience=20, restore_best_weights=True)
def schedule(epoch):
    if epoch < 80:
        return 0.0005
    elif epoch < 160:
        return 0.0001
    else:
        return 0.00001
lrs = LearningRateScheduler(schedule)

In [29]:
def DenseBlock(n, activation='relu'):
    def f(x):
        x = layers.Dense(n, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.advanced_activations.LeakyReLU(0.3)(x)
        
        return x
    return f


In [44]:
K.clear_session()

inputs = layers.Input(shape=(train_X.shape[1], ))

x1 = layers.Dense(256*4, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Activation('relu')(x1)

x1 = DenseBlock(256*4)(x1)
x1 = DenseBlock(256*3)(x1)
x1 = DenseBlock(256*2)(x1)
x1 = DenseBlock(256)(x1)
x1 = DenseBlock(64)(x1)
x1 = DenseBlock(32)(x1)

x2 = layers.Dense(256*4, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x2 = layers.BatchNormalization()(x2)
x2 = layers.Activation('relu')(x2)

x2 = DenseBlock(256, 'tanh')(x2)
# x2 = DenseBlock(256, 'tanh')(x2)
x2 = DenseBlock(128, 'tanh')(x2)
# x2 = DenseBlock(64, 'tanh')(x2)
x2 = DenseBlock(32, 'tanh')(x2)


x = layers.concatenate([x1, x2])

x = layers.Dense(32, activation='relu')(x)

x = layers.Dense(19, activation='softmax')(x1)

m = models.Model(inputs, x)

# sparse_categorical_crossentropy
m.compile(optimizer = 'adam',
         loss = 'sparse_categorical_crossentropy', 
          metrics = ['acc'])

In [46]:
m.fit(train_X, t, 
     epochs = 1000,
     validation_split=0.1,
     batch_size=1024*16,
      callbacks=[es, lrs]
     )

Train on 179991 samples, validate on 20000 samples
Epoch 1/1000
179991/179991 [==============================] - 10s 56us/step - loss: 8.3359 - acc: 0.1225 - val_loss: 8.2081 - val_acc: 0.2394
Epoch 2/1000
179991/179991 [==============================] - 8s 44us/step - loss: 7.8404 - acc: 0.2676 - val_loss: 7.9121 - val_acc: 0.2966
Epoch 3/1000
179991/179991 [==============================] - 8s 46us/step - loss: 7.4368 - acc: 0.3331 - val_loss: 7.5921 - val_acc: 0.2704
Epoch 4/1000
179991/179991 [==============================] - 8s 46us/step - loss: 7.0529 - acc: 0.3761 - val_loss: 7.2896 - val_acc: 0.2863
Epoch 5/1000
179991/179991 [==============================] - 8s 45us/step - loss: 6.6868 - acc: 0.4046 - val_loss: 6.9720 - val_acc: 0.3093
Epoch 6/1000
179991/179991 [==============================] - 8s 46us/step - loss: 6.3883 - acc: 0.4299 - val_loss: 6.6380 - val_acc: 0.3968
Epoch 7/1000
179991/179991 [==============================] - 8s 46us/step - loss: 6.0755 - acc: 0.458

179991/179991 [==============================] - 8s 43us/step - loss: 1.4828 - acc: 0.7448 - val_loss: 2.9412 - val_acc: 0.4029
Epoch 59/1000
179991/179991 [==============================] - 8s 43us/step - loss: 1.4480 - acc: 0.7514 - val_loss: 2.9160 - val_acc: 0.4419
Epoch 60/1000
179991/179991 [==============================] - 8s 43us/step - loss: 1.4206 - acc: 0.7536 - val_loss: 3.0107 - val_acc: 0.4532
Epoch 61/1000
179991/179991 [==============================] - 8s 43us/step - loss: 1.4151 - acc: 0.7526 - val_loss: 2.9851 - val_acc: 0.4643
Epoch 62/1000
179991/179991 [==============================] - 8s 43us/step - loss: 1.4115 - acc: 0.7486 - val_loss: 2.9182 - val_acc: 0.4401
Epoch 63/1000
179991/179991 [==============================] - 8s 43us/step - loss: 1.3973 - acc: 0.7481 - val_loss: 2.9399 - val_acc: 0.4624
Epoch 64/1000
179991/179991 [==============================] - 8s 43us/step - loss: 1.3789 - acc: 0.7526 - val_loss: 2.8868 - val_acc: 0.4944
Epoch 65/1000
179991

179991/179991 [==============================] - 7s 38us/step - loss: 1.0102 - acc: 0.8083 - val_loss: 2.7288 - val_acc: 0.5020
Epoch 116/1000
179991/179991 [==============================] - 7s 38us/step - loss: 1.0007 - acc: 0.8122 - val_loss: 2.7175 - val_acc: 0.4999
Epoch 117/1000
179991/179991 [==============================] - 7s 38us/step - loss: 0.9967 - acc: 0.8129 - val_loss: 2.8229 - val_acc: 0.4683
Epoch 118/1000
179991/179991 [==============================] - 7s 38us/step - loss: 1.0042 - acc: 0.8085 - val_loss: 2.8439 - val_acc: 0.4481
Epoch 119/1000
179991/179991 [==============================] - 7s 38us/step - loss: 0.9957 - acc: 0.8118 - val_loss: 2.8381 - val_acc: 0.4697
Epoch 120/1000
179991/179991 [==============================] - 7s 39us/step - loss: 0.9948 - acc: 0.8102 - val_loss: 2.8115 - val_acc: 0.4797
Epoch 121/1000
179991/179991 [==============================] - 7s 38us/step - loss: 0.9953 - acc: 0.8092 - val_loss: 2.8075 - val_acc: 0.4751
Epoch 122/1000

In [14]:
m.evaluate(train_X, t)

199991/199991 [==============================] - 25s 125us/step


[3.7297216974958327, 0.37130171060562134]

In [47]:
m.evaluate(train_X, t)

199991/199991 [==============================] - 24s 120us/step


[2.6539636167787592, 0.5075178146362305]

In [48]:
kk = pd.DataFrame(m.predict(test_X), columns=target_lbe.classes_)
sub[sub.columns[1:]] = kk[sub.columns[1:]]

In [49]:
sub.head()

,id,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
0,199991,2.252655e-14,6.449129e-19,0.000020,0.999979,2.086145e-18,5.856213e-07,7.357396e-16,1.351230e-13,6.133254e-17,1.687138e-14,1.214196e-08,4.484849e-13,6.288822e-15,8.487848e-23,8.539606e-18,1.869033e-16,1.593677e-14,3.758303e-18,6.081606e-16
1,199992,1.412883e-04,1.040885e-04,0.028695,0.032322,1.881269e-04,1.161975e-03,6.795549e-01,7.349603e-05,1.469379e-01,4.502118e-02,3.377729e-05,5.452726e-02,4.188034e-04,4.764224e-05,8.069664e-03,7.227518e-04,1.953870e-03,1.577347e-05,1.052274e-05
2,199993,1.210330e-03,4.051838e-03,0.000026,0.000185,1.551057e-05,3.586822e-04,9.539144e-01,1.512179e-04,1.114309e-03,8.051617e-04,8.948028e-05,2.053663e-02,3.409594e-04,3.621213e-03,9.717903e-03,7.651238e-04,2.399540e-03,1.621220e-04,5.349171e-04
3,199994,2.262035e-02,2.587308e-02,0.000745,0.002052,2.668079e-07,8.989718e-05,3.884218e-03,7.160466e-01,9.083239e-02,4.593516e-03,2.574253e-04,2.948985e-03,1.016943e-02,8.810913e-04,1.536400e-05,3.180351e-05,9.801646e-03,3.147640e-04,1.088424e-01
4,199995,1.954893e-12,5.048577e-15,0.003604,0.996390,7.911671e-14,5.667650e-06,6.553776e-11,4.025667e-12,2.463272e-13,8.296262e-11,5.587081e-08,4.994810e-10,1.170522e-12,1.398107e-19,8.713635e-14,3.482315e-13,1.206919e-11,1.377900e-15,5.722839e-15


In [50]:
sub.to_csv('./sub/nn4.csv', index=False)

In [11]:
print(train_X.shape, test_X.shape)

(199991, 1020) (10009, 1020)
